First things first let's initialise our environment.

*Run the cell below to import our libraries and initialise our LUSID client*

In [61]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
from dateutil.parser import parse
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import math
import json

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('API Version: ', client.api_version)

LUSID Environment Initialised
API Version:  0.9.187


![Initailise LUSID](img/multiplecurrencies-initialise.gif)

## 1) Set up our Scope

In [2]:
scope_id = str(uuid.uuid4())[:4]
scope_name = 'UK_High_Growth_Equities_Fund'
scope = '{}_{}'.format(scope_name, scope_id)
print (scope)

UK_High_Growth_Equities_Fund_2f59


![Scope](img/multiplecurrencies-scope.gif)

## 2) Load Instruments

In [37]:
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
instrument_universe.head(n=10)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,BBG00FN3B5K8,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.000
3,USTreasury_6.875_2025,imd_34534539,USD,US912810EV62,BBG000DQQNJ8,NEW YORK,united_states_america,T 6.875 08/15/25,govt,us_government,6.875
4,ExpressScripts_NYSE_ESRX,imd_34352311,USD,US30219G1085,BBG000C16621,UN,united_states_america,ESRX,equity,common_stock,NaN
5,TrinityIndustries_NYSE_TRN,imd_34235200,USD,US8965221091,BBG000BVL406,UN,united_states_america,TRN,equity,common_stock,NaN
6,Trex_NYSE_TREX,imd_32423956,USD,US89531P1057,BBG000BTGM43,UN,united_states_america,TREX,equity,common_stock,NaN
7,Cigna_NYSE_CI,imd_32452391,USD,US1255091092,BBG00KXXK940,UN,united_states_america,CI,equity,common_stock,NaN
8,Arcosa_NYSE_ACA,imd_23423409,USD,US0396531008,BBG00JGMWFQ5,UN,united_states_america,ACA,equity,common_stock,NaN


In [38]:
# Initialise our dictionary to hold our instrument definitions
instrument_definitions = {}

# Iterate over our instrument universe
for index, instrument in instrument_universe.iterrows():

    # Create our ISIN property using the default ISIN property definition
    isin_property = models.InstrumentProperty(
        key='Instrument/default/Isin',
        value=models.PropertyValue(
            label_value=instrument['isin']))
    
    # Create our ticker property using the default ticker property definition
    ticker_property = models.InstrumentProperty(
        key='Instrument/default/Ticker',
        value=models.PropertyValue(
            label_value=instrument['ticker']))

    # Create the definition for our instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers={
            'Figi': instrument['figi'],
            'ClientInternal': instrument['client_internal']},
        properties=[
            isin_property,
            ticker_property
        ]
    )

# Call LUSID to upsert our instrument defintions
response = client.upsert_instruments(requests=instrument_definitions)

# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
Figi ID: BBG000BVPXP1
LUSID Instrument ID: LUID_GJBAEYMY


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
Figi ID: BBG000B9XVV8
LUSID Instrument ID: LUID_Z6LRHXAF


Instrument Successfully Upserted: Trex_NYSE_TREX
Figi ID: BBG000BTGM43
LUSID Instrument ID: LUID_TSQ51Z7Z


Instrument Successfully Upserted: Arcosa_NYSE_ACA
Figi ID: BBG00JGMWFQ5
LUSID Instrument ID: LUID_B4F63NZM


Instrument Successfully Upserted: USTreasury_6.875_2025
Figi ID: BBG000DQQNJ8
LUSID Instrument ID: LUID_OZ5XG0E5


Instrument Successfully Upserted: ExpressScripts_NYSE_ESRX
Figi ID: BBG000C16621
LUSID Instrument ID: LUID_X8K7UME5


Instrument Successfully Upserted: Cigna_NYSE_CI
Figi ID: BBG00KXXK940
LUSID Instrument ID: LUID_QWB8KLXQ


Instrument Successfully Upserted: USTreasury_2.00_2021
Figi ID: BBG00FN3B5K8
LUSID Instrument ID: LUID_3ABWW5FM


Instrument Successfully Upserted: TrinityIndustries_NYSE_TRN
Figi ID: BBG000BVL406
LUSID Instrument ID: LU

![Instruments](img/multiplecurrencies-instruments.gif)

## 3) Create our Base Fund Portfolio

In [5]:
base_portfolio_name = '{}_base_fund'.format(scope_name)
print (base_portfolio_name)

UK_High_Growth_Equities_Fund_base_fund


In [6]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=base_portfolio_name,
    code=base_portfolio_name,
    base_currency='USD',
    description='The portfolio to hold our base fund',
    created=portfolio_creation_date)

# Call LUSID to create our portfolio
response = client.create_portfolio(
    scope=scope,
    create_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund
Portfolio Effective From: 2018-03-22 11:15:49.345297+00:00
Portfolio Created On: 2019-03-22 11:15:49.018791+00:00



![Base-Fund](img/multiplecurrencies-basefundportfolio.gif)

## 4) Set our Holdings

In [7]:
#Import our holdings
holdings = pd.read_csv('data/multiplecurrencies-holdings.csv')
holdings.head()

,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency
0,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,1550.00,7750000.00,USD
1,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,49567,190.00,9417730.00,USD
2,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,99.25,12063142.75,USD
3,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,98444,140.98,13878635.12,USD


In [8]:
holding_adjustments = []

holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=4)

for row, holding in holdings.iterrows():

    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/Figi': holding['figi']},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['total_cost'],
                            currency=holding['currency']),
                        portfolio_cost=holding['total_cost'],
                        price=holding['unit_cost'])
                ]
        )
    )
    
# Call LUSID to set our holdings
response = client.set_holdings(
    scope=scope,
    code=base_portfolio_name,
    effective_at=holdings_effective_date,
    holding_adjustments=holding_adjustments)

prettyprint.set_holdings_response(response, scope, base_portfolio_name)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund
Holdings Effective From: 2019-03-18 11:15:50.539557+00:00
Holdings Created On: 2019-03-22 11:15:50.220804+00:00



![Base-Fund-Hodlings](img/multiplecurrencies-basefundportfolioholdings.gif)

## 5) Add Corporate Actions

In [9]:
corporate_action_source_id = '{}_corporate_action_source'.format(base_portfolio_name)

corporate_action_request = models.CreateCorporateActionSourceRequest(
    scope=scope,
    code=corporate_action_source_id)

response = client.create_corporate_action_source(
    request=corporate_action_request)
    
prettyprint.corporate_action_response(response)

Corporate Actions Source Successfully Created
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_corporate_action_source


![Corporate-Action-Source](img/multiplecurrencies-corporateactionsource.gif)

In [10]:
corporate_actions = pd.read_csv('data/corporateactions.csv')
corporate_actions.head()

,code,action_description,description,announcement_date,ex_date,record_date,payment_date,input_instrument_figi,instrument_input_ticker,input_instrument_name,input_units_factor,input_cost_factor,output_instrument_figi,output_instrument_name,output_ticker,output_instrument_internal,output_units_factor,output_cost_factor,dividend_yield
0,5943592342,dividend,Apple Dividend,29/01/2019,08/02/2019,11/02/2019,14/02/2019,BBG000B9XVV8,AAPL,APPLE INC,1,1,NaN,NaN,NaN,CCY_GBP,0.0170,0.0000,0.017
1,5943592343,merger,Cigna & Express Scripts Merger,08/03/2018,10/12/2018,13/12/2018,20/12/2018,BBG000C16621,ESRX,EXPRESS SCRIPTS HOLDINGS CO,1,1,NaN,NaN,NaN,CCY_USD,48.7500,0.0000,NaN
2,5943592343,merger,Cigna & Express Scripts Merger,08/03/2018,10/12/2018,13/12/2018,20/12/2018,BBG000C16621,ESRX,EXPRESS SCRIPTS HOLDINGS CO,1,1,BBG00KXXK940,CIGNA CORP,CI,NaN,0.2434,1.9337,NaN
3,5943592347,spin-off,Acosa spin-off from Trinity Industries,12/12/2017,17/10/2018,20/10/2018,01/11/2018,BBG000BVL406,TRN,TRINITY INDUSTRIES INC,3,1,BBG00JGMWFQ5,ARCOSA INC,ACA,NaN,1.0000,1.0324,NaN
4,5943592498,split,Trex 2 for 1 stock split,07/05/2018,21/05/2018,23/05/2018,19/06/2018,BBG000BTGM43,TREX,TREX COMPANY INC,1,1,BBG000BTGM43,TREX COMPANY INC,TREX,NaN,2.0000,2.0000,NaN


In [11]:
actions = {}


for row, action in corporate_actions.iterrows():
    
    actions[action['code']] = {}
    
    actions[action['code']].setdefault(
        'input_transition', models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                'Instrument/default/Figi': action['input_instrument_figi']
            },
            units_factor=action['input_units_factor'],
            cost_factor=action['input_cost_factor']))
    
    actions[action['code']].setdefault(
        'output_transitions', []).append(
            models.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    'Instrument/default/Figi': action['input_instrument_figi']
                },
                units_factor=action['input_units_factor'],
                cost_factor=action['input_cost_factor']))
    
actions_list = []

for action_id, action in actions.items():
    
    transition = models.CorporateActionTransitionRequest(
        input_transition=actions[action_id]['input_transition'],
        output_transitions=actions[action_id]['output_transitions']
    )
    
    actions_list.append(
        models.CreateCorporateAction(
            corporate_action_code=action_id,
            announcement_date=datetime.now(pytz.UTC)+timedelta(days=1),
            ex_date=datetime.now(pytz.UTC)+timedelta(days=1),
            record_date=datetime.now(pytz.UTC)+timedelta(days=1),
            payment_date=datetime.now(pytz.UTC)+timedelta(days=1),
            transitions=[transition])
    )

response = client.batch_upsert_corporate_actions(
    scope=scope,
    code=corporate_action_source_id,
    actions=actions_list)

prettyprint.corporate_actions_added_response(response)

Corporate Action Successfully Upserted
Code: 5943592342
Announcement Date: 2019-03-23 11:15:52.404765+00:00
Ex Date: 2019-03-23 11:15:52.404784+00:00

Corporate Action Successfully Upserted
Code: 5943592343
Announcement Date: 2019-03-23 11:15:52.404808+00:00
Ex Date: 2019-03-23 11:15:52.404813+00:00

Corporate Action Successfully Upserted
Code: 5943592347
Announcement Date: 2019-03-23 11:15:52.404830+00:00
Ex Date: 2019-03-23 11:15:52.404834+00:00

Corporate Action Successfully Upserted
Code: 5943592498
Announcement Date: 2019-03-23 11:15:52.404850+00:00
Ex Date: 2019-03-23 11:15:52.404854+00:00



In [12]:
response = client.upsert_portfolio_details(
    scope=scope,
    code=base_portfolio_name,
    details=models.CreatePortfolioDetails(
        corporate_action_source_id=models.ResourceId(
            scope=scope,
            code=corporate_action_source_id),
        base_currency='GBP'))

prettyprint.portfolio_details_response(response)

Portfolio Details: 
Detail Origin Portfolio Scope: UK_High_Growth_Equities_Fund_2f59
Detail Origin Portfolio Code: UK_High_Growth_Equities_Fund_base_fund
Base Currency: GBP
Corporate Action Source Id Scope: UK_High_Growth_Equities_Fund_2f59
Corporate Action Source Id Code: UK_High_Growth_Equities_Fund_base_fund_corporate_action_source




![Corporate-Actions](img/multiplecurrencies-corporateactions.gif)

## 6) Value our Base Fund Portfolio

### a) Upsert Market Quotes

In [13]:
prices = pd.read_csv('data/multiplecurrencies-prices.csv')
prices.head()

,price,type,currency,instrument_name,instrument_figi,instrument_internal
0,1622.65,close,USD,Amazon_Nasdaq_AMZN,BBG000BVPXP1,imd_34634534
1,170.80,close,USD,Apple_Nasdaq_AAPL,BBG000B9XVV8,imd_34535347
2,98.00,close,USD,USTreasury_2.00_2021,BBG00FN3B5K8,imd_34535347
3,142.36,close,USD,USTreasury_6.875_2025,BBG000DQQNJ8,imd_34534539


In [14]:
instrument_quotes = []

for row, quote in prices.iterrows():

    instrument_quotes.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                instrument_id=quote['instrument_figi'],
                instrument_id_type='Figi',
                quote_type='Price',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=quote['price'],
                unit=quote['currency']),
            quote_lineage=models.QuoteLineage(
                data_vendor='Yahoo_Finance',
                contributor='Internal_User'),
            effective_at=holdings_effective_date
            )
    )

response = client.upsert_quotes(
    scope=scope,
    quotes=instrument_quotes)

prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-03-22 11:15:57.508109+00:00


![Analytic-Stores](img/multiplecurrencies-quotestore.gif)

### b) Value using Quotes via Aggregation

In [15]:
inline_recipe = models.ConfigurationRecipe(
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.Figi.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid'),
           models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid'),
            models.MarketDataKeyRule(
               key='Fx.CurrencyPair.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Rate',
               price_side='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='Figi',
            default_scope=scope)
    )
)

aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/SubHoldingKey',
        op='Value'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Units',
        op='sum'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='sum'),
    ],
    group_by=[
        'Holding/default/SubHoldingKey'
    ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

prettyprint.aggregation_response_generic(response)

Aggregation Results: 
Holding/default/SubHoldingKey: LusidInstrumentId=LUID_GJBAEYMY/USD
Instrument/default/Name: Amazon_Nasdaq_AMZN
Sum(Holding/default/Units): 5000.0
Sum(Holding/default/Cost): 7750000.0
Sum(Holding/default/PV): 8113250.0


Holding/default/SubHoldingKey: LusidInstrumentId=LUID_Z6LRHXAF/USD
Instrument/default/Name: Apple_Nasdaq_AAPL
Sum(Holding/default/Units): 49567.0
Sum(Holding/default/Cost): 9417730.0
Sum(Holding/default/PV): 8466043.6


Holding/default/SubHoldingKey: LusidInstrumentId=LUID_3ABWW5FM/USD
Instrument/default/Name: USTreasury_2.00_2021
Sum(Holding/default/Units): 121543.0
Sum(Holding/default/Cost): 12063142.75
Sum(Holding/default/PV): 11911214.0


Holding/default/SubHoldingKey: LusidInstrumentId=LUID_OZ5XG0E5/USD
Instrument/default/Name: USTreasury_6.875_2025
Sum(Holding/default/Units): 98444.0
Sum(Holding/default/Cost): 13878635.12
Sum(Holding/default/PV): 14014487.84




![Analytic-Stores](img/multiplecurrencies-aggregatedbase.gif)

## 7) Securitise our Base Fund Portfolio

### a) Securitiese with x units

In [16]:
# Create our request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=scope,
    code='total_circulation',
    value_required=False,
    display_name='total_circulation',
    data_type_id=models.ResourceId(scope='default', code='number'))

# Call LUSID to create our new property
response = client.create_property_definition(definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
circulation_property_key = response.key

# Pretty print our key
prettyprint.heading('Circulation Property Key', circulation_property_key)

Circulation Property Key: Instrument/UK_High_Growth_Equities_Fund_2f59/total_circulation


![Analytic-Stores](img/multiplecurrencies-circulationproperty.gif)

In [28]:
instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': base_portfolio_name},
    properties=[models.InstrumentProperty(
        key=circulation_property_key,
        value=models.PropertyValue(
            metric_value=models.MetricValue(
                value=40000.00
                )))],
    look_through_portfolio_id=models.ResourceId(
        scope=scope,
        code=base_portfolio_name))

response = client.upsert_instruments(
    requests={
        'look-through':instrument_definition
    }
)

prettyprint.instrument_response(response, identifier='ClientInternal')

Instrument Successfully Upserted: look-through
ClientInternal ID: UK_High_Growth_Equities_Fund_base_fund
LUSID Instrument ID: LUID_1E4YQGP1


1  instruments upserted successfully
0  instrument upsert failures


![Analytic-Stores](img/multiplecurrencies-securitisebase.gif)

### b) Calculalate price from aggregation and store in the quote store

In [35]:
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/Cost',
        op='sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='sum'),
        models.AggregateSpec(key='Portfolio/default/Name',
        op='sum'),
    ],
    group_by=[
        'Portfolio/default/Name'
    ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

prettyprint.aggregation_response_generic(response)

base_fund_total_value = response.data[0]['Sum(Holding/default/PV)']
base_fund_total_cost = response.data[0]['Sum(Holding/default/Cost)']

Aggregation Results: 
Sum(Holding/default/Cost): 43109507.87
Sum(Holding/default/PV): 42504995.44




In [42]:
base_fund_instrument = client.get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_name,
    instrument_property_keys=[circulation_property_key])

luid = base_fund_instrument.lusid_instrument_id
base_fund_total_circulation = round(base_fund_instrument.properties[0].value, 0)
base_fund_unit_price = round(base_fund_total_value / base_fund_total_circulation, 2)

instrument_quote = models.UpsertQuoteRequest(
    quote_id=models.QuoteId(
        instrument_id=luid,
        instrument_id_type='LusidInstrumentId',
        quote_type='Price',
        price_side='Mid'),
    metric_value=models.MetricValue(
        value=base_fund_unit_price,
        unit='USD'),
    quote_lineage=models.QuoteLineage(
        data_vendor='Internal',
        contributor='Internal_User'),
    effective_at=holdings_effective_date
)

response = client.upsert_quotes(
    scope=scope,
    quotes=[instrument_quote])

prettyprint.heading('Base Fund Total Circulation', str(base_fund_total_circulation))
prettyprint.heading('Base Fund Total Value', str(base_fund_total_value))
prettyprint.heading('Base Fund Unit Price', str(base_fund_unit_price))
prettyprint.upsert_quotes_response(response)

Base Fund Total Circulation: 40000.0
Base Fund Total Value: 42504995.44
Base Fund Unit Price: 1062.62
Quotes Successfully Upserted At: 2019-03-22 11:57:48.191570+00:00


![Base-Fund-Price](img/multiplecurrencies-basefundprice.gif)

## 8) Create Foreign Currency Portfolios

In [20]:
fund_currencies = ['AUD', 'GBP', 'USD', 'EUR', 'JPY']

for currency in fund_currencies:
    
    portfolio_name = '{}_{}'.format(base_portfolio_name, currency)
    
    # Create the request to add our portfolio
    transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_name,
        code=portfolio_name,
        base_currency=currency,
        description='The portfolio to hold our base fund in currency {}'.format(currency),
        created=portfolio_creation_date)

    # Call LUSID to create our portfolio
    portfolio_response = client.create_portfolio(
        scope=scope,
        create_request=transaction_portfolio_request)

    # Pretty print the response from LUSID
    prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_AUD
Portfolio Effective From: 2018-03-22 11:15:49.345297+00:00
Portfolio Created On: 2019-03-22 11:16:00.156280+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_GBP
Portfolio Effective From: 2018-03-22 11:15:49.345297+00:00
Portfolio Created On: 2019-03-22 11:16:01.265739+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_USD
Portfolio Effective From: 2018-03-22 11:15:49.345297+00:00
Portfolio Created On: 2019-03-22 11:16:02.367286+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_EUR
Portfolio Effective From: 2018-03-22 11:15:49.345297+00:00
Portfolio Created On: 2019-03-22 11:16:03.432291+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_JPY


![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfolios.gif)

## 9) Add Base Fund Units

In [21]:
fx_rates = pd.read_csv('data/multiplecurrencies-fxrates.csv')
fx_rates.head()

,pair,rate
0,GBP/USD,1.310
1,AUD/USD,0.720
2,EUR/USD,1.140
3,JPY/USD,0.009


In [43]:
quote_requests = []

for index, fx_rate in fx_rates.iterrows():
    if 'USD/USD' != fx_rate['pair']:
        quote_requests.append(
            models.UpsertQuoteRequest(
                quote_id=models.QuoteId(
                    instrument_id=fx_rate['pair'],
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid'),
                metric_value=models.MetricValue(
                    value=fx_rate['rate'],
                    unit='rate'),
                quote_lineage=models.QuoteLineage(
                    data_vendor='Internal',
                    contributor='Internal_User'),
                effective_at=holdings_effective_date))

        reverse_pair = '/'.join(fx_rate['pair'].split('/')[::-1])

        quote_requests.append(
            models.UpsertQuoteRequest(
                quote_id=models.QuoteId(
                    instrument_id=reverse_pair,
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid'),
                metric_value=models.MetricValue(
                    value=1/fx_rate['rate'],
                    unit='rate'),
                quote_lineage=models.QuoteLineage(
                    data_vendor='internal',
                    contributor='internal_user'),
                effective_at=holdings_effective_date))
    
response = client.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-03-22 11:58:14.791400+00:00


In [23]:
base_fund_units_per_portfolio = base_fund_total_circulation/len(fund_currencies)
total_cost_per_portfolio = base_fund_total_value/len(fund_currencies)

for currency in fund_currencies:
    
    if currency!='USD':
        fx_rate = client.get_quotes(
            scope=scope,
            quote_ids=[
                models.QuoteId(
                    instrument_id='{}/USD'.format(currency),
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid')
            ],
            effective_at=holdings_effective_date,
            as_at=None,
            max_age=None,
            page=None,
            limit=None).found[0].metric_value.value
    else:
        fx_rate = 1
    
    holding_adjustments = []
    
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/ClientInternal': base_portfolio_name},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=base_fund_units_per_portfolio,
                        cost=models.CurrencyAndAmount(
                            amount=total_cost_per_portfolio,
                            currency='USD'),
                        portfolio_cost=total_cost_per_portfolio/fx_rate, 
                        price=base_fund_total_value/base_fund_total_circulation)
                ]
        )
    )

    # Call LUSID to set our holdings
    response = client.set_holdings(
        scope=scope,
        code='{}_{}'.format(base_portfolio_name, currency),
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)

    prettyprint.set_holdings_response(response, scope, '{}_{}'.format(base_portfolio_name, currency))

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_AUD
Holdings Effective From: 2019-03-18 11:15:50.539557+00:00
Holdings Created On: 2019-03-22 11:16:06.318779+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_GBP
Holdings Effective From: 2019-03-18 11:15:50.539557+00:00
Holdings Created On: 2019-03-22 11:16:07.246726+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_USD
Holdings Effective From: 2019-03-18 11:15:50.539557+00:00
Holdings Created On: 2019-03-22 11:16:07.879277+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_EUR
Holdings Effective From: 2019-03-18 11:15:50.539557+00:00
Holdings Created On: 2019-03-22 11:16:10.120842+00:00

Holdings Successfully Set fo

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfoliosholdbase.gif)

## 10) Add Hedging Transactions

In [44]:
hedge_contracts = {}

for currency in fund_currencies:
    
    if currency!='USD':
        fx_rate = client.get_quotes(
            scope=scope,
            quote_ids=[
                models.QuoteId(
                    instrument_id='{}/USD'.format(currency),
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid')
            ],
            effective_at=holdings_effective_date,
            as_at=None,
            max_age=None,
            page=None,
            limit=None).found[0].metric_value.value
    else:
        fx_rate = 1
    
    
    hedge_contract = {
        "contract_id": "tid_{}".format(uuid.uuid4()),
        "type": "FwdFxSell",
        "legs": 2,
        "transaction_date": holdings_effective_date.isoformat(),
        "settlement_date": (holdings_effective_date+timedelta(days=7)).isoformat(),
        "strike_price": 1/fx_rate,
        "leg1": {
            "type": "sell",
            "currency": "USD",
            "amount": round(total_cost_per_portfolio, 0)
        },
        "leg2": {
            "type": "buy",
            "currency": "GBP",
            "amount": round(total_cost_per_portfolio, 0) / fx_rate
        }
    }
    
     # Create the definition for our instrument
    hedge_contracts['{}_hedge'.format(currency)] = models.InstrumentDefinition(
        name='hedge_{}'.format(currency),
        identifiers={
            'ClientInternal': hedge_contract["contract_id"]},
        definition=models.InstrumentEconomicDefinition(
            instrument_format='JSON',
            content=json.dumps(hedge_contract)
        )
    )
        
# Call LUSID to upsert our instrument defintions
response = client.upsert_instruments(requests=hedge_contracts)

# Pretty print the response from LUSID
prettyprint.instrument_response(response)

Instrument Successfully Upserted: EUR_hedge
ClientInternal ID: tid_025f124d-a96d-460b-9888-02694f94a61c
LUSID Instrument ID: LUID_1HLULAGS


Instrument Successfully Upserted: GBP_hedge
ClientInternal ID: tid_a2347a5f-8ff6-4350-8f08-5b64050f396b
LUSID Instrument ID: LUID_9VF4TDYM


Instrument Successfully Upserted: JPY_hedge
ClientInternal ID: tid_56354169-ef17-46b4-b2aa-b373126f53db
LUSID Instrument ID: LUID_MGJR7CP0


Instrument Successfully Upserted: USD_hedge
ClientInternal ID: tid_be9ca402-1818-4e2d-920a-e2e8c967baa9
LUSID Instrument ID: LUID_JKILYPAA


Instrument Successfully Upserted: AUD_hedge
ClientInternal ID: tid_61f461ba-0a35-49d2-8b2e-8bfc6e3404e0
LUSID Instrument ID: LUID_8MNZJO6Y


5  instruments upserted successfully
0  instrument upsert failures


In [25]:
 for currency in fund_currencies:
        
    hedge_transactions = []

    contract = json.loads(client.get_instrument(
        identifier_type='ClientInternal',
        identifier=hedge_contracts['{}_hedge'.format(currency)].identifiers['ClientInternal']
    ).instrument_definition.content)

    hedge_transactions.append(
        models.TransactionRequest(
            transaction_id=contract['contract_id'],
            type='Buy',
            instrument_identifiers={
                'Instrument/default/ClientInternal': contract['contract_id']},
            transaction_date='T'.join(contract['transaction_date'].split(' ')),
            settlement_date='T'.join(contract['settlement_date'].split(' ')),
            units=1,
            transaction_price=models.TransactionPrice(
                price=0,
                type='Price'),
            total_consideration=models.CurrencyAndAmount(
                amount=0,
                currency=currency),
            transaction_currency='USD',
            source='Client',
        )
    )
    
    
    response = client.upsert_transactions(
        scope=scope,
        code='{}_{}'.format(base_portfolio_name, currency),
        transactions=hedge_transactions
        )
  
    prettyprint.transactions_response(
        response, 
        scope, 
        '{}_{}'.format(base_portfolio_name, currency))

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_AUD
Transactions Effective From: 2019-03-18 11:15:50.539557+00:00
Transactions Created On: 2019-03-22 11:16:14.866669+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_GBP
Transactions Effective From: 2019-03-18 11:15:50.539557+00:00
Transactions Created On: 2019-03-22 11:16:15.721559+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_USD
Transactions Effective From: 2019-03-18 11:15:50.539557+00:00
Transactions Created On: 2019-03-22 11:16:16.881051+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_2f59
Code: UK_High_Growth_Equities_Fund_base_fund_EUR
Transactions Effective From: 2019-03-18 11:15:50.539557+00:00
Transactions Cr

## 11) Value Currency Portfolios

## Value the Currency Forward Contracts

In [26]:
interest_rates = pd.read_csv('data/multiplecurrencies-interestrates.csv')
interest_rates.head()

,currency,interest_rate
0,USD,0.025500
1,AUD,0.017100
2,EUR,-0.001611
3,JPY,0.001000
4,GBP,0.007719


In [45]:
quote_requests = []

for index, interest_rate in interest_rates.iterrows():
    
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                instrument_id=interest_rate['currency'] + '_RiskFreeRate',
                instrument_id_type='ClientInternal',
                quote_type='Rate',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=interest_rate['interest_rate'],
                unit='rate'),
            quote_lineage=models.QuoteLineage(
                data_vendor='Yahoo_Finance',
                contributor='default'),
            effective_at=holdings_effective_date))
    
response = client.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

prettyprint.upsert_quotes_response(response)

ErrorResponseException: There was a problem with the request

In [54]:
quote_requests = []

for currency in fund_currencies:
    
    if currency!='USD':
        fx_rate = client.get_quotes(
            scope=scope,
            quote_ids=[
                models.QuoteId(
                    instrument_id='{}/USD'.format(currency),
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid')
            ],
            effective_at=holdings_effective_date,
            as_at=None,
            max_age=None,
            page=None,
            limit=None).found[0].metric_value.value
    else:
        fx_rate = 1
    
    response = client.get_transactions(
        scope=scope,
        code='{}_{}'.format(base_portfolio_name, currency),
        from_transaction_date=holdings_effective_date,
        to_transaction_date=holdings_effective_date,
        as_at=None,
        sort_by=None,
        start=None,
        limit=None,
        instrument_property_keys=None,
        filter="type eq 'Buy'")
    
    if len(response.values) == 0:
        continue
        
    hedge_transaction = response.values[0]
    
    hedge_contract = json.loads(client.get_instrument(
        identifier_type='LusidInstrumentId',
        identifier=hedge_transaction.instrument_uid
    ).instrument_definition.content)
    
    spot_rate = 1 / fx_rate
    
    interest_rate_1 = interest_rates.loc[
        interest_rates['currency'] == 'USD', 'interest_rate'].values[0]
    
    interest_rate_2 = interest_rates.loc[
        interest_rates['currency'] == currency, 'interest_rate'].values[0]
    
    time = (parse(timestr=hedge_contract['settlement_date']) \
            - datetime.now(pytz.UTC)).days / 365
    
    strike_price = hedge_contract['strike_price']

    forward_price = spot_rate * math.exp((interest_rate_2 - interest_rate_1) * time)
    value = (forward_price - strike_price) * math.exp(-interest_rate_2 * time)
    total_value = value * hedge_contract['leg1']['amount']
    
    prettyprint.heading(
        'USD/' + currency+' swap value', '{} {}'.format(
            round(total_value,2), currency))
    
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                instrument_id=hedge_transaction.instrument_uid,
                instrument_id_type='LusidInstrumentId',
                quote_type='Price',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=total_value,
                unit=currency),
            quote_lineage=models.QuoteLineage(
                data_vendor='Internal',
                contributor='default'),
            effective_at=holdings_effective_date))
    
response = client.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

prettyprint.upsert_quotes_response(response)

USD/AUD swap value: -543.38 AUD
USD/GBP swap value: -632.2 GBP
USD/USD swap value: 0.0 USD
USD/EUR swap value: -1107.71 EUR
USD/JPY swap value: -126794.13 JPY
Quotes Successfully Upserted At: 2019-03-22 12:02:41.471509+00:00


## Value the Currency Portfolios

In [71]:
currency_portfolio_values = {}

for currency in fund_currencies:
    
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe,
            effective_at=holdings_effective_date,
            metrics=[
                models.AggregateSpec(key='Holding/default/PV',
                op='sum'),
                models.AggregateSpec(key='Portfolio/default/Name',
                op='value'),

            ],
            group_by=[
                'Portfolio/default/Name'
            ])

    response = client.get_aggregation_by_portfolio(
        scope=scope,
        code=base_portfolio_name+'_{}'.format(currency),
        request=aggregation_request)
    
    currency_portfolio_values[currency] = response.data[0]['Sum(Holding/default/PV)']
    
    prettyprint.aggregation_response_generic(response)

ClientRequestError: Error occurred in request., RetryError: HTTPSConnectionPool(host='api.lusid.com', port=443): Max retries exceeded with url: /api/portfolios/UK_High_Growth_Equities_Fund_2f59/UK_High_Growth_Equities_Fund_base_fund_AUD/$aggregate (Caused by ResponseError('too many 500 error responses',))

## 12) Securitise Currency Portfolios

In [67]:
instrument_definitions = {}

for currency in fund_currencies:

    instrument_definitions['currency_{}'.format(currency)] = (models.InstrumentDefinition(
        name=base_portfolio_name,
        identifiers={'ClientInternal': '{}_{}'.format(base_portfolio_name, currency)},
        properties=[models.InstrumentProperty(
            key=circulation_property_key,
            value=models.PropertyValue(
                metric_value=models.MetricValue(
                    value=10000
                    )))],
        look_through_portfolio_id=models.ResourceId(
            scope=scope,
            code='{}_{}'.format(base_portfolio_name, currency)
        )
    ))

response = client.upsert_instruments(
    requests=instrument_definitions
)

prettyprint.instrument_response(response, identifier='ClientInternal')

Instrument Successfully Upserted: currency_GBP
ClientInternal ID: UK_High_Growth_Equities_Fund_base_fund_GBP
LUSID Instrument ID: LUID_MR3GPHI6


Instrument Successfully Upserted: currency_EUR
ClientInternal ID: UK_High_Growth_Equities_Fund_base_fund_EUR
LUSID Instrument ID: LUID_E58ESRAZ


Instrument Successfully Upserted: currency_JPY
ClientInternal ID: UK_High_Growth_Equities_Fund_base_fund_JPY
LUSID Instrument ID: LUID_IEC1HP9T


Instrument Successfully Upserted: currency_USD
ClientInternal ID: UK_High_Growth_Equities_Fund_base_fund_USD
LUSID Instrument ID: LUID_ZPVLBSQ8


Instrument Successfully Upserted: currency_AUD
ClientInternal ID: UK_High_Growth_Equities_Fund_base_fund_AUD
LUSID Instrument ID: LUID_10TLNOY7


5  instruments upserted successfully
0  instrument upsert failures


In [70]:
instrument_quotes = []

for currency in fund_currencies:

    currency_fund_instrument = client.get_instrument(
        identifier_type='ClientInternal',
        identifier='{}_{}'.format(base_portfolio_name, currency),
        instrument_property_keys=[circulation_property_key])
    
    luid = base_fund_instrument.lusid_instrument_id
    
    currency_fund_total_circulation = round(
        base_fund_instrument.properties[0].value, 0)
    
    currency_fund_unit_price = round(
        currency_portfolio_values[currency] / base_fund_total_circulation, 2)

    instrument_quotes.append(models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            instrument_id=luid,
            instrument_id_type='LusidInstrumentId',
            quote_type='Price',
            price_side='Mid'),
        metric_value=models.MetricValue(
            value=currency_fund_unit_price,
            unit=currency),
        quote_lineage=models.QuoteLineage(
            data_vendor='Internal',
            contributor='Internal_User'),
        effective_at=holdings_effective_date
    ))
    
    prettyprint.heading(f'{currency} Fund Total Circulation', str(currency_fund_total_circulation))
    prettyprint.heading(f'{currency} Fund Total Value', str(currency_fund_total_value))
    prettyprint.heading(f'{currency} Fund Unit Price', str(currency_fund_unit_price))

response = client.upsert_quotes(
    scope=scope,
    quotes=instrument_quotes)

prettyprint.upsert_quotes_response(response)

KeyError: 'AUD'

## 13) Create Share Class Portfolios

## 14) Value Each Share Class

## 15) Add Transactions to the Base Portfolio

## 16) Value each share class portfolio (bubble up)

## 17) Add Hedging to one of the currency portfolios

## 18) Value each share class portfolio (bubble up)